## 구글드라이브 마운팅

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

## 셀레니움 실행

In [3]:
# # colab에서 셀레니움 실행하기
# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [4]:
# # 런타임 재시작
# import os
# os.kill(os.getpid(), 9)

## 모듈 import

In [1]:
# 모듈 import
import time
import sys 
import os 
import re
import pdb

import pandas as pd
import numpy as np

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys

from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

## path 설정

In [42]:
# 코랩
# driver_path = "/content/drive/My Drive/projects/NLP-StockMarket/chromedriver.exe"
# csv_path = "/content/drive/My Drive/projects/NLP-StockMarket/"
# os.chdir(csv_path)

In [12]:
# 주피터
driver_path = "C:/chromedriver.exe"
csv_path = "C:/users/user/"

os.chdir(csv_path)

In [44]:
# #Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')        # Head-less 설정
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome('chromedriver', options=options)

## 기간 설정

In [6]:
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

In [46]:
# test
# input_date_list = date_range("20210101", "20210102")
# input_date_list

## 경제홈의 증권 뉴스

In [17]:
def article_Scraping():
    input_date_list = date_range(start, end)
    article_title_list = []
    article_date_list = []
    cols = ['Data', 'Title']
    driver = webdriver.Chrome(driver_path)
    result_df = pd.DataFrame(columns = cols)

    for date in input_date_list:
        past_li = []
        page_cnt = 1
        while True :
            li = []
            driver.get(f'https://news.naver.com/main/list.naver?mode=LS2D&sid2=258&sid1=101&mid=shm&date={date}&page={page_cnt}')
            time.sleep(0.5)
            articles1 = driver.find_elements_by_css_selector(f'.type06 > li > dl > dt > a')
            articles2 = driver.find_elements_by_css_selector(f'.type06_headline > li > dl > dt > a')
            articles_li = [articles1, articles2]
            time.sleep(1)
            for articles in articles_li:
                for article in articles:
                    if article.text == '동영상기사':
                        pass
                    elif len(article.text) != 0:
                        article_date_list.append(date)
                        article_title_list.append(article.text)
                    li.append(article.text)
#                     pdb.set_trace()
                        
            time.sleep(0.5)
            
            if past_li == li :
                break
            else :
                past_li = li
                page_cnt += 1
            if len(li) < NEWS_MAX_CNT :
                break
                
        crawling_df = pd.DataFrame(list(zip(article_date_list, article_title_list)), columns = cols)
        crawling_df.drop_duplicates(['Title'], keep='first', inplace=True)
    driver.quit()
    result_df = pd.concat([result_df, crawling_df])
    result_df.drop_duplicates(['Title'], keep='first', inplace=True)
    return result_df

In [18]:
NEWS_MAX_CNT = 20

In [19]:
start = "20220101"
end = "20220531"

In [20]:
result_df = article_Scraping()
result_df

,Data,Title
0,20220101,1월엔 어떤株 사야하나
1,20220101,"[마켓워치] 새해 첫 주 미국 고용지표 관심, 국내는 종부세 경감안 나올 듯"
2,20220101,소재부터 친환경까지…기업가치 120兆 넘보는 SK
3,20220101,"한 달새 10%뛴 항공株, 오미크론에도 반등 신호"
4,20220101,"[주간 추천주] ""IT·모빌리티에 관심""…LG이노텍·SK이노 '러브콜'"
...,...,...
110436,20220531,연말 부실금융기관 딱지 붙을라…보험사 노심초사
110437,20220531,[머니 컨설팅]금리인상기엔 ELS-국채 투자 고려를
110438,20220531,[주요경제지표] 2022년 5월 30일자
110439,20220531,[주식 매매 상위 종목 및 환율] 2022년 5월 30일자


In [25]:
# result_df.to_csv('naver_news_crawling_20210101to20211231.csv')
result_df.to_csv('naver_news_crawling_20220101to20220531.csv', encoding = 'utf-8-sig')

## 증권 홈 뉴스

In [145]:
def article_Scraping2():
    input_date_list = date_range("20210101", "20211231")
    article_title_list = []
    article_date_list = []
    cols = ['Data', 'news_title']
    driver = webdriver.Chrome(driver_path)
    result_df = pd.DataFrame(columns = cols)
    for date in input_date_list:
        for page in range(1, 16):  # 최대 15(아마도?)하고 중복된 항목 제거필요
#         for page in range(1, 5):
            try:
                driver.get(f'https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=402&date={date}&page={page}')
                time.sleep(0.5)
                articles = driver.find_elements_by_css_selector(f'.articleSubject > a')
                time.sleep(1)
                for article in articles:
                    if article.text == '동영상기사':
                        pass
                    elif len(article.text) != 0:
                        article_date_list.append(date)
                        article_title_list.append(article.text)                
            except:
                pass
        crawling_df = pd.DataFrame(list(zip(article_date_list, article_title_list)), columns = cols)
        crawling_df.drop_duplicates(['news_title'], keep='first', inplace=True)
    driver.quit()
    result_df = pd.concat([result_df, crawling_df])
    result_df.drop_duplicates(['news_title'], keep='first', inplace=True)
    return result_df

In [146]:
result_df2 = article_Scraping2()
result_df2

,Date,Title
0,20220626,"""쪼개기 상장 피해 없게"" 주주 보호 대책 쏟아내는 기업들"
1,20220626,"하나금융투자, 하나증권으로 새출발"
2,20220626,환노출 ETF “强달러 덕봤다”
3,20220626,외국인 탈출에 반대매매 가중... 실물지표 따라 변동성 커질듯 [주간 증시 전망]
4,20220626,네이버·카카오 성장성에 베팅… 우량채로 리스크 관리까지 [이런 펀드 어때요?]
...,...,...
102,20220627,"NCT 127, 日 첫 돔투어 화려한 피날레 '총 22만 관객 동원'"
103,20220627,'1위' 와이너리와 보틀벙커가 만났다…트라피체 시음 행사
104,20220627,신재생에너지 바람에도 원전주 '강세'…왜?
105,20220627,"DB금융투자 ""고물가에 투자 대안은 배당…KT&G 등 배당귀족주"""


In [22]:
new_df = pd.read_csv('naver_news_crawling_20210101to20211231.csv', index_col=0, encoding = 'utf-8-sig')
new_df

,Data,news_title
0,20210101,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까"
1,20210101,목표가 속속 100만원대…`흥`오른 삼성바이오로직스
2,20210101,신생 PEF운용사 `안타` 떡볶이업체 SJ코레 사들여 1년만에 매각…80억 차익
3,20210101,삼성전자 신고가 랠리…'9만전자' 가능할까
4,20210101,"작년 매출 신기록 쓴 카카오·엔씨…""올해 더 좋다"""
...,...,...
451345,20211231,이달만 6000억 샀다…'박스피'에 지친 개미들 몰린 '이곳'
451346,20211231,불타오른 IPO 시장…내년 대어급 총출동에 기대↑
451347,20211231,올해초 수준 회복 못하고 마무리한 낙폭과대株는?
451348,20211231,"유럽 증시, 산타 랠리 기대감으로 상승 마감"


In [23]:
new_df.columns = ['Date', 'Title']
new_df

,Date,Title
0,20210101,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까"
1,20210101,목표가 속속 100만원대…`흥`오른 삼성바이오로직스
2,20210101,신생 PEF운용사 `안타` 떡볶이업체 SJ코레 사들여 1년만에 매각…80억 차익
3,20210101,삼성전자 신고가 랠리…'9만전자' 가능할까
4,20210101,"작년 매출 신기록 쓴 카카오·엔씨…""올해 더 좋다"""
...,...,...
451345,20211231,이달만 6000억 샀다…'박스피'에 지친 개미들 몰린 '이곳'
451346,20211231,불타오른 IPO 시장…내년 대어급 총출동에 기대↑
451347,20211231,올해초 수준 회복 못하고 마무리한 낙폭과대株는?
451348,20211231,"유럽 증시, 산타 랠리 기대감으로 상승 마감"


In [24]:
new_df.to_csv('naver_news_crawling_20210101to20211231_re.csv', encoding = 'utf-8-sig')